In [1]:
import pandas as pd

red_wine_df = pd.read_csv('red-wine.csv')
red_wine_df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [2]:
# Create training and validation splits
df_train = red_wine_df.sample(frac=0.7, random_state=0)
df_valid = red_wine_df.drop(df_train.index)
df_train.head(4)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
1109,10.8,0.470,0.43,2.10,0.171,27.0,66.0,0.99820,3.17,0.76,10.8,6
1032,8.1,0.820,0.00,4.10,0.095,5.0,14.0,0.99854,3.36,0.53,9.6,5
1002,9.1,0.290,0.33,2.05,0.063,13.0,27.0,0.99516,3.26,0.84,11.7,7
487,10.2,0.645,0.36,1.80,0.053,5.0,14.0,0.99820,3.17,0.42,10.0,6


In [3]:
# Scale to [0, 1]
max_ = df_train.max(axis=0)
min_ = df_train.min(axis=0)
df_train = (df_train - min_) / (max_ - min_)
df_valid = (df_valid - min_) / (max_ - min_)

In [4]:
# Split features and target
X_train = df_train.drop('quality', axis=1)
X_valid = df_valid.drop('quality', axis=1)
y_train = df_train['quality']
y_valid = df_valid['quality']

In [5]:
X_train.shape

(1119, 11)

In [6]:
from tensorflow import keras
from tensorflow.keras import layers, callbacks

In [27]:
early_stopping = callbacks.EarlyStopping(
    min_delta=0.002, # minimium amount of change to count as an improvement
    patience=100, # how many epochs to wait before stopping
    restore_best_weights=True,
)

In [8]:
model = keras.Sequential([
    layers.Dense(units=300, activation='relu', input_shape=[11]),
    layers.Dropout(rate=0.3),# to break wrong clustering of computations for better analysis
    layers.BatchNormalization(),# to normalize the batch data that comes in the layer
    
    layers.Dense(units=400, activation='relu'),
    layers.Dropout(rate=0.3),
    layers.BatchNormalization(),
    
    layers.Dense(units=400, activation='relu'),
    layers.Dropout(rate=0.3),
    layers.BatchNormalization(),
    
    layers.Dense(units=200, activation='relu'),
    layers.Dropout(rate=0.3),
    layers.BatchNormalization(),
    
    layers.Dense(units=1)
])

In [9]:
model.compile(
    optimizer="adam", # tunes the model to find the least loss at every step
    loss="mae", # calculates the difference between the true values and predicted values
)

In [28]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_valid, y_valid),
    batch_size=50,
    epochs=200,
    
    callbacks=[early_stopping], # put your callbacks in a list
    #verbose=0,  # turn off training log
)

Epoch 1/200
23/23 [==============================] - 0s 13ms/step - loss: 0.0850 - val_loss: 0.0886
Epoch 2/200
23/23 [==============================] - 0s 10ms/step - loss: 0.0881 - val_loss: 0.0880
Epoch 3/200
23/23 [==============================] - 0s 10ms/step - loss: 0.0886 - val_loss: 0.0926
Epoch 4/200
23/23 [==============================] - 0s 11ms/step - loss: 0.0886 - val_loss: 0.0877
Epoch 5/200
23/23 [==============================] - 0s 11ms/step - loss: 0.0906 - val_loss: 0.0874
Epoch 6/200
23/23 [==============================] - 0s 11ms/step - loss: 0.0876 - val_loss: 0.0904
Epoch 7/200
23/23 [==============================] - 0s 10ms/step - loss: 0.0875 - val_loss: 0.0885
Epoch 8/200
23/23 [==============================] - 0s 10ms/step - loss: 0.0888 - val_loss: 0.0917
Epoch 9/200
23/23 [==============================] - 0s 10ms/step - loss: 0.0876 - val_loss: 0.0884
Epoch 10/200
23/23 [==============================] - 0s 10ms/step - loss: 0.0864 - val_loss: 0.0891